# Ensemble Learning & Random Forests

## Table of Contents

---

### 1. [Ensemble Learning](#Ensemble-Learning)
### 2. [Voting Classifiers](#Voting-Classifiers)
### 3. [Bagging & Pasting](#Bagging-&-Pasting)
### 4. [Boosting](#Boosting)
### 5. [Stacking](#Stacking)

---


## Setup

In [7]:
import pandas as pd

# To split dataset for testing and training
from sklearn.model_selection import train_test_split

# Dataset used for binary classification
from sklearn.datasets import make_moons

# Metrics
from sklearn.metrics import accuracy_score

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

## Ensemble Learning

Ensemble learning is when you **put together** trained models instead of picking just one.

### There are four main ensemble methods:
### 1. Voting
### 2. Bagging/Pasting
### 3. Boosting
### 4. Stacking

## Voting Classifiers

![voting image](./images/voting.jpg)

- Train **many models** on **all** the training data, then select the best
- Let the models *'vote'* on the classification
- The **majority**, or most agreed on answer is your classification

In [2]:
# Ensemble Model for Voting
from sklearn.ensemble import VotingClassifier

# Individual Models
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

### We are going to make a voting classifier that looks like this:
![Diagram of Voting Classifier Configuration](./images/votingclf_config.PNG)

In [65]:
# Three Individual Classification Models
logistic_classifier = LogisticRegression(solver="liblinear", random_state=42)
random_forest_classifier = RandomForestClassifier(n_estimators=10, random_state=42)
support_vector_classifier = SVC(gamma="auto", random_state=42, probability=True)

In [66]:
# One Voting Classifier to Aggregate Them
hard_voting_classifier = VotingClassifier(
    estimators = [
        ('lr', logistic_classifier), 
        ('rf', random_forest_classifier), 
        ('svc', support_vector_classifier)
    ],
    voting = 'hard' # hard OR soft
)

soft_voting_classifier = VotingClassifier(
    estimators = [
        ('lr', logistic_classifier), 
        ('rf', random_forest_classifier), 
        ('svc', support_vector_classifier)
    ],
    voting = 'soft'
)

## Two Types of Voting
### Hard
- Take the highest number of votes

### Soft
- Add up the probabilities to pick the one that we are most sure about

#### Example of Hard Voting
![Diagram of Voting Classifier Configuration](./images/votingclf.PNG)


In [78]:
voting_results = pd.DataFrame()

for clf in ( logistic_classifier, random_forest_classifier, support_vector_classifier, 
            hard_voting_classifier, soft_voting_classifier):
    
    # Train our models on the training data
    clf.fit(X_train, y_train)
    
    # Evaluate their performance using the testing data
    y_pred = clf.predict(X_test)
    
    name = type(clf).__name__
    if name == 'VotingClassifier':
        name = name + ' ' + clf.voting
    
    # Store our accuracy results in a dataframe
    model_result = { 'Model': name, 'Accuracy': accuracy_score(y_test, y_pred) }
    voting_results = voting_results.append(model_result, ignore_index=True)

Text(0, 0.5, 'Model\n')

In [89]:
colors = ['r', 'b', 'purple', 'darkturquoise']

barchart = voting_results.sort_values(by='Accuracy', ascending=True).plot.barh('Model', 
                                    'Accuracy', 
                                    color=colors, 
                                    fontsize=20, 
                                    title='Voting Classifier Results',
                                    legend=False,
                                    figsize=(10,10))

barchart.set_xlabel('\nAccuracy', fontsize=20)
barchart.set_ylabel('Model\n', fontsize=20)

In [90]:
voting_results.sort_values(by='Accuracy', ascending=False)

,Accuracy,Model
4,0.912,VotingClassifier soft
5,0.904,BaggingClassifier
3,0.896,VotingClassifier hard
2,0.888,SVC
1,0.872,RandomForestClassifier
0,0.864,LogisticRegression
6,0.856,DecisionTreeClassifier


## Bagging & Pasting

- Select **one** model type
- Train many models of that type on **random subsets** of the data
- Let the models *'vote'* on the classification (soft or hard like our voting classifiers)
- The **majority**, or most agreed on answer is your classification

### Terminology

**Bagging (Bootstrap Aggregating)**: When the sampled data is put back, giving other models a chance to sample it

**Pasting** When the sampled data is removed, so no model see's it more than once

### So lets train a bagging classifier on our data

In [81]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

# Pick our model type for the bagging classifier
decision_tree_classifier = DecisionTreeClassifier(random_state=42)

# Define our Bagging Classifier
bag_clf = BaggingClassifier(
    decision_tree_classifier,
    n_estimators=500,
    max_samples=100, 
    bootstrap=True, 
    n_jobs=-1, 
    random_state=42)

# Train the ensemble model, and our lone model to see how it compares
bag_clf.fit(X_train, y_train)
decision_tree_classifier.fit(X_train, y_train)

# Evaluate their accuracy
y_pred_bag = bag_clf.predict(X_test)
y_pred_control = decision_tree_classifier.predict(X_test)

In [82]:
# Store our accuracy results in a dataframe
model_result = { 'Model': type(bag_clf).__name__, 'Accuracy': accuracy_score(y_test, y_pred_bag) }
voting_results = voting_results.append(model_result, ignore_index=True)

model_result = { 'Model': type(decision_tree_classifier).__name__, 'Accuracy': accuracy_score(y_test, y_pred_control) }
voting_results = voting_results.append(model_result, ignore_index=True)

Text(0, 0.5, 'Model\n')

In [92]:
barchart = voting_results.sort_values(by='Accuracy', ascending=True).plot.barh('Model', 
                                    'Accuracy', 
                                    color=colors, 
                                    fontsize=20, 
                                    title='Voting Classifier Results',
                                    legend=False,
                                    figsize=(10,10))

barchart.set_xlabel('\nAccuracy', fontsize=20)
barchart.set_ylabel('Model\n', fontsize=20)

In [94]:
voting_results.sort_values(by='Accuracy', ascending=False)

,Accuracy,Model
4,0.912,VotingClassifier soft
5,0.904,BaggingClassifier
3,0.896,VotingClassifier hard
2,0.888,SVC
1,0.872,RandomForestClassifier
0,0.864,LogisticRegression
6,0.856,DecisionTreeClassifier


### Random Forests

### The bagging classifier we just made is a special type of ensemble. 
A **Random Forest Tree** is an ensemble of decision tree's made from bagging or pasting. We made one using the built in RandomForestClassifier class in the voting section as well.

## Boosting

- Train a model
- Identify the models mistakes
- Feed the mistakes to a new model that is trained to fix the last ones mistakes
- Repeat, chaining together a sequence of improving models

## Stacking

### Why choose hard or soft voting when we can train a model to make the decision?

In stacking you feed all the model outputs from an ensemble method like bagging or voting into another model that is trained to pick the correct answer based on the other models behavior.

![Diagram of Voting Classifier Configuration](./images/stacking.PNG)

### Stacking isn't supported in Scikit Learn yet so I don't have an example for you!